In [1]:
import time
import datetime
import requests

In [2]:
session = requests.Session()
with open ("token.txt", "r") as f:
    x = f.readline()
session.headers.update({"Authorization": "token "+x.strip()})

In [3]:
api_request_limit_per_hour = 4999
error_url = []
limit_check_hour = datetime.datetime.now().hour
can_continue = True

def HasItPassLimitHour(current_hour):
    return limit_check_hour != current_hour

def CheckRequestLimit():
    global api_request_limit_per_hour
    if api_request_limit_per_hour != 0:
        can_continue = True
    else:
        current_hour = datetime.datetime.now().hour
        if HasItPassLimitHour(current_hour) == True:
            limit_check_hour = datetime.datetime.now().hour
            api_request_limit_per_hour = 4999
            can_continue = True
        else:
            can_continue = False
    
def FetchDataFromGitHubRestApi(url):
    global api_request_limit_per_hour
    CheckRequestLimit()
    while can_continue == False:
        time.sleep(60)
        CheckRequestLimit()
    response = session.get(url)
    api_request_limit_per_hour -= 1
    if response.status_code == 200:
        fetched_data = response.json()
        poll_interval = None
        poll_interval = response.headers.get('X-Poll-Interval')
        if poll_interval:
            time.sleep(int(poll_interval))
        return fetched_data
    else:
        error_url.append(url)

In [4]:
username = "kritserv"

In [5]:
profile_url = "https://api.github.com/users/"+username
profile_data = FetchDataFromGitHubRestApi(profile_url)

In [6]:
event_data = {}

for i in (1,2,3):
    event_url_pagination = "https://api.github.com/users/"+username+"/events?per_page=100&page="+str(i)
    event_data["page"+str(i)] = FetchDataFromGitHubRestApi(event_url_pagination)

In [7]:
event_data["page1"][0]

{'id': '34712153108',
 'type': 'PushEvent',
 'actor': {'id': 122365475,
  'login': 'kritserv',
  'display_login': 'kritserv',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/kritserv',
  'avatar_url': 'https://avatars.githubusercontent.com/u/122365475?'},
 'repo': {'id': 738980587,
  'name': 'kritserv/analyze-python-user-on-github',
  'url': 'https://api.github.com/repos/kritserv/analyze-python-user-on-github'},
 'payload': {'repository_id': 738980587,
  'push_id': 16579665716,
  'size': 1,
  'distinct_size': 1,
  'ref': 'refs/heads/main',
  'head': 'd3f8497308bd8a5ad8791bc65263c16b4fe07cd1',
  'before': '6bdbb0be775d48f3ed68bd94b95d6140293d7a82',
  'commits': [{'sha': 'd3f8497308bd8a5ad8791bc65263c16b4fe07cd1',
    'author': {'email': 'krittipoom@outlook.com', 'name': 'kritserv'},
    'message': 'improve comment in auto scroll script',
    'distinct': True,
    'url': 'https://api.github.com/repos/kritserv/analyze-python-user-on-github/commits/d3f8497308bd8a5ad8791bc65263c

In [8]:
for page in event_data:
    for event in event_data[page]:
        print(event['type'])

PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
CreateEvent
CreateEvent
PushEvent
PullRequestEvent
PullRequestEvent
PushEvent
PushEvent
PullRequestEvent
PullRequestEvent
PushEvent
PushEvent
PullRequestEvent
PullRequestEvent
PushEvent
PushEvent
PullRequestEvent
PullRequestEvent
PushEvent
CreateEvent
PushEvent
PushEvent
PushEvent
ForkEvent
PushEvent
PushEvent
WatchEvent
WatchEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
CreateEvent
CreateEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
CreateEvent
CreateEvent
PushEven

In [9]:
len(event_data)

3

In [10]:
len(event_data["page1"]), len(event_data["page2"]), len(event_data["page3"])

(100, 100, 99)

In [11]:
api_request_limit_per_hour

4995

In [12]:
starred_data = FetchDataFromGitHubRestApi("https://api.github.com/users/"+username+"/starred?per_page=100&sort=updated")

In [13]:
len(starred_data)

36

In [14]:
for star_repo in starred_data:
    if star_repo["language"]:
        pass
    else:
        star_repo["language"] = "None"
    print("full name: "+star_repo["full_name"], 
          "html_url: "+star_repo["html_url"], 
          "description: "+star_repo["description"].replace(',',''), 
          "language: "+star_repo["language"], 
          "topics: "+str(star_repo['topics']).replace("'",'').replace(", ",'/').replace('[','').replace(']',''), 
          "stargazers_count: "+str(star_repo["stargazers_count"]), sep='\n', end='\n\n')

full name: bigskysoftware/htmx
html_url: https://github.com/bigskysoftware/htmx
description: </> htmx - high power tools for HTML
language: JavaScript
topics: hateoas/html/htmx/hyperscript/javascript/rest
stargazers_count: 26595

full name: mongodb/mongo
html_url: https://github.com/mongodb/mongo
description: The MongoDB Database
language: C++
topics: c-plus-plus/database/mongodb/nosql
stargazers_count: 24914

full name: gohugoio/hugo
html_url: https://github.com/gohugoio/hugo
description: The world’s fastest framework for building websites.
language: Go
topics: blog-engine/cms/content-management-system/documentation-tool/go/hugo/static-site-generator
stargazers_count: 70774

full name: django/django
html_url: https://github.com/django/django
description: The Web framework for perfectionists with deadlines.
language: Python
topics: apps/django/framework/models/orm/python/templates/views/web
stargazers_count: 74976

full name: renpy/renpy
html_url: https://github.com/renpy/renpy
descrip

In [15]:
print('login: '+profile_data['login'], 
      'type: '+profile_data['type'], 
      'location: '+profile_data['location'].replace(',',''), 
      'hireable: '+str(profile_data['hireable']), 
      'public_repos: '+str(profile_data['public_repos']), 
      'public_gists: '+str(profile_data['public_gists']), 
      'followers: '+str(profile_data['followers']), 
      'following: '+str(profile_data['following']), 
      'created_at: '+profile_data['created_at'], 
      'updated_at: '+profile_data['updated_at'], sep='\n')

login: kritserv
type: User
location: Bangkok Thailand
hireable: True
public_repos: 14
public_gists: 1
followers: 3
following: 5
created_at: 2023-01-10T12:07:55Z
updated_at: 2024-01-06T14:57:09Z


In [16]:
repo_data = FetchDataFromGitHubRestApi("https://api.github.com/users/"+username+"/repos?per_page=100&sort=updated")

In [17]:
for repo in repo_data:
    if repo["language"]:
        pass
    else:
        repo["language"] = "None"
    if repo["description"]:
        pass
    else:
        repo["description"] = "None"
    print("description: "+repo['description'], 
          "fork: "+str(repo['fork']), 
          "stargazers_count: "+str(repo['stargazers_count']), 
          "language: "+repo['language'], 
          "topics: "+str(repo['topics']).replace("'",'').replace(", ",'/').replace('[','').replace(']',''), 
          "forks: "+str(repo['forks']), sep='\n', end='\n\n')

description: API for the tarot cards of the Rider-Waite-Smith deck using go
fork: True
stargazers_count: 0
language: Go
topics: api/go/golang/tarot-cards
forks: 0

description: A python code that prints the result of a neural network trying to solve Pythagoras’ theorem in real-time in your terminal.
fork: False
stargazers_count: 0
language: Python
topics: neural-network/pythagoras/python
forks: 0

description: None
fork: False
stargazers_count: 0
language: HTML
topics: 
forks: 0

description: None
fork: False
stargazers_count: 0
language: HTML
topics: 
forks: 0

description: None
fork: False
stargazers_count: 0
language: HTML
topics: 
forks: 0

description: Ecommerce website build with Django, Docker, SQLite.
fork: False
stargazers_count: 0
language: Python
topics: django/django-project/docker/ecommerce-website/python
forks: 0

description: None
fork: False
stargazers_count: 0
language: Python
topics: 
forks: 0

description: A python notebook for fake and real voice (from FOR Dataset) 